In [8]:
import numpy as np

# Implementacja NSGA-II

In [17]:
class NSGAII:
    def __init__(self, chromosome_length, population_size, initial_population, initial_objective_values):
        self.chromosome_length = chromosome_length
        self.population_size = population_size
        if type(initial_population) is not np.ndarray:
            raise ValueError("initial population must be a numpy array")
        if initial_population.shape != (population_size, chromosome_length):
            raise ValueError("initial population must be of shape (population_size, chromosome_length)")
        self.current_population = initial_population
        if type(initial_objective_values) is not np.ndarray:
            raise ValueError("initial objective values must be a numpy array")
        if initial_population.shape != (population_size, chromosome_length):
            raise ValueError("initial population values must be of shape (population_size, objectives_number)")
        self.current_objective_values = initial_objective_values

    def get_population(self):
        return self.current_population

    def non_dominated_sort(self, population_values):

        def dominates(vals1, vals2):
            return np.all(vals1 >= vals2) and np.any(vals1 > vals2)

        pareto_frontier = np.empty(self.population_size, dtype=int)
        current_frontier_number = 1
        current_frontier_members = []
        # number of solutions dominating p
        dominating_count = np.zeros(self.population_size, dtype=int)
        # set of solutions dominated by p
        dominated = [None] * self.population_size 

        for p in range(self.population_size):
            dominated[p] = []
            for q in range(self.population_size):
                if dominates(p, q):
                    dominated[p].append(q)
                elif dominates(q, p):
                    dominating_count[p] += 1
            if dominating_count[p] == 0:
                pareto_frontier[p] = current_frontier_number
                current_frontier_members.append(p)

        while current_frontier_members != []:
            current_frontier_number += 1
            new_frontier_members = []
            for p in current_frontier_members:
                for q in dominated[p]:
                    dominating_count[q] -= 1
                    if dominating_count[q] == 0:
                        pareto_frontier[q] = current_frontier_number
                        new_frontier_members.append(q)
            current_frontier_members = new_frontier_members
        
        return pareto_frontier

        

    def generate_children(self, objective_values, number_of_offspring = self.population_size):
        # TODO
        # parent selection
        # recombination
        # mutation
        pass

    def replace_population(self, children_population, children_objective_values):
        # TODO
        # combine current population and children population
        # sort it using non dominated sort
        self.current_population = combined_population[:population_size]
        self.current_objective_values = combined_objective_values[:population_size]


NameError: ignored

In [13]:
l = [1]
l == []

False